In [1]:
from model_bet import Graph_Diff_Reg
import networkx as nx
import torch
import torch.nn as nn
import torch.optim as optim
import random, glob, time, os, pickle
import numpy as np
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

from sklearn.metrics import r2_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('The following device is used: '+str(device))

The following device is used: cuda


In [13]:
hidden = 256
num_epoch = 200

n_features= 256#list_fm_train_shuffled[0].shape[1]
dropout_val = 0.4
learning_rate = 0.00005

model = Graph_Diff_Reg(input_dim=n_features,hidden_dim=hidden,output_dim=1,dropout=dropout_val)
model = model.to(device)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.00005/10)

training_loss_epochs = []
training_mse_epochs = []

testing_loss_epochs= []
testing_mse_epochs= []

training_score_epochs = []
testing_score_epochs = []

In [14]:
def train(filefolder):
    model.train() 
    
    loss_train = 0
    num_files_train = len(filefolder)
    
    starttime3 = time.time()
    y_trues=[]
    y_preds=[]
    
    random.shuffle(filefolder)
    n=0
    
    for i in range(num_files_train):
        print("Batch: "+str(i))

        filepath=filefolder[i]
        with open(filepath,"rb") as f:
            # list_adj_deg_train_shuffled, list_adj_wt_train_shuffled,list_num_edge_train, bc_mat_train_shuffled, list_fm_train_shuffled, list_n_seq_train, list_n_unshuffle_train,list_adj_deg_test_shuffled, list_adj_wt_test_shuffled,list_num_edge_test, bc_mat_test_shuffled, list_fm_test_shuffled, list_n_seq_test, list_n_unshuffle_test=pickle.load(f)
            list_train_graph, list_edge_index_train,list_edge_weight_train, eff_mat_train, list_fm_train_orig,list_edge_index_train_orig,list_edge_weight_train_orig=pickle.load(f)
        
        # for j in range(len(list_train_graph)):
        #     print(len(list_train_graph[j].edges()))
        
        
        for j in range(len(list_train_graph)):
            
            # g=list_train_graph
            # adj2=torch.tensor(nx.adjacency_matrix(list_train_graph[j]).toarray()).numpy()
            
            # for 
            
            
            edge_index=list_edge_index_train[j]
            edge_weigth=list_edge_weight_train[j]
            
            # adj=torch.tensor(edge_list_to_adjacency_matrix(edge_index.numpy().transpose(), edge_weigth.numpy(), 500))
            # adj=adj.to(device)
            
            edge_index=edge_index.to(device)
            edge_weigth=edge_weigth.to(device)
            fm=list_fm_train_orig[0]
            # fm=torch.tensor(np.eye(500,dtype=np.float32))
            
            batch_vector=np.zeros((fm.shape[0]),dtype=np.int64)
            batch_vector=torch.tensor(batch_vector)
            batch_vector=batch_vector.to(device)
            
            fm=fm.to(device).to_dense()
            # adj_orig=torch.tensor(nx.adjacency_matrix(list_train_graph_orig[j]).toarray())
            edge_index_orig=list_edge_index_train_orig[0]
            edge_weigth_orig=list_edge_weight_train_orig[0]
            
            # adj_orig=torch.tensor(edge_list_to_adjacency_matrix(edge_index_orig.numpy().transpose(), edge_weigth_orig.numpy(), 500))
            # adj_orig=adj_orig.to(device)
            
            # adjn=adj.cpu().numpy()
            # adjon=adj_orig.cpu().numpy()
            # adjr=adjon-adjn
            # adjr[adjr!=0]=1
            # adjr=torch.tensor(adjr.astype(np.float32)).to(device)
            
            # edge_index_n=edge_index.cpu().numpy()
            # edge_index_orig_n=edge_index_orig.cpu().numpy()
            edge_weight_n=edge_weigth.cpu().numpy()
            edge_weight_n[edge_weight_n<10000]=1
            edge_weight_n[edge_weight_n!=1]=0
            edge_mask=torch.tensor(edge_weight_n.astype(np.int64)).to(device)
            
            edge_index_orig=edge_index_orig.to(device)
            
            edge_weigth_orig=edge_weigth_orig.to(device)
            
            y_true=torch.tensor(eff_mat_train[0,j].reshape([1,1]).astype(np.float32))
            y_true=y_true.to(device)
            
            # adj = list_adj_deg_train_shuffled[j]
            # num_nodes = list_num_edge_train[j]
            # adj_t = list_adj_wt_train_shuffled[j]
            # fm = list_fm_train_shuffled[j]
            # adj = adj.to(device)
            # adj_t = adj_t.to(device)
            # fm=fm.to(device)
            
            optimizer.zero_grad()
                
            # y_out = model(adj,adj_t,fm)
            y_out = model(edge_index,edge_weigth.float(),edge_index_orig,edge_weigth_orig.float(),fm.float(),batch_vector)#,adjr.float(),edge_mask)#,adj,adj_orig)
                        
            # true_arr = torch.from_numpy(bc_mat_train_shuffled[:,j]).float()
            # true_val = true_arr.to(device)
            
            # true_val_final = true_val[list_n_unshuffle_train[j]]
            # y_out_final = y_out[list_n_unshuffle_train[j]]
            
            # true_val_ultimate = true_val_final[:num_nodes]
            # y_out_ultimate = y_out_final[:num_nodes]
            
            mse = loss_fn(y_out, y_true)
            
            mse.backward()
            
            
            
            optimizer.step()
            
            y_trues.append(float(y_true))
            y_preds.append(float(y_out))
            
            
            # loss_rank = new_loss_cal(y_out_ultimate, true_val_ultimate, num_nodes, device, model_size)
            loss_train = loss_train + float(mse)
            n+=1
            
    mse_final=loss_train/n
    duration3 = time.time()-starttime3
    print("training time this epoch:"+str(duration3))
    print("training loss:"+str(loss_train))
    print("training rmse:"+str(np.sqrt(float(mse_final))))
    
    score=r2_score(y_trues, y_preds)
    print("Training_R_Squared:"+str(score))
    
    return loss_train,mse_final,score

In [15]:
filefolder = glob.glob('./graph_data_train.pickle')
print("Training")
print(f"Total Number of epoches: {num_epoch}")
for e in range(num_epoch):
    print(f"Epoch number: {e+1}/{num_epoch}")
    print("LR: "+str(learning_rate))
  
    
    train_loss,train_mse,train_score = train(filefolder)
    training_loss_epochs.append(train_loss)
    training_mse_epochs.append(train_mse)
    training_score_epochs.append(train_score)

Training
Total Number of epoches: 200
Epoch number: 1/200
LR: 5e-05
Batch: 0
training time this epoch:3.6668589115142822
training loss:37.57090714174274
training rmse:0.6129511166621914
Training_R_Squared:-9.126560765832174
Epoch number: 2/200
LR: 5e-05
Batch: 0
training time this epoch:2.1442008018493652
training loss:22.098883172905516
training rmse:0.4700944923406944
Training_R_Squared:-4.956355764515544
Epoch number: 3/200
LR: 5e-05
Batch: 0
training time this epoch:1.9184436798095703
training loss:9.376490368431405
training rmse:0.3062105544952918
Training_R_Squared:-1.5272640129401864
Epoch number: 4/200
LR: 5e-05
Batch: 0
training time this epoch:1.7919204235076904
training loss:5.780198813694369
training rmse:0.2404204403476204
Training_R_Squared:-0.5579484119609235
Epoch number: 5/200
LR: 5e-05
Batch: 0
training time this epoch:1.5983502864837646
training loss:5.375439678059365
training rmse:0.23184994453437693
Training_R_Squared:-0.4488528781266652
Epoch number: 6/200
LR: 5e-

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x000002010A2F31C0>>
Traceback (most recent call last):
  File "c:\Users\Anderson\miniconda3\envs\gnn\lib\site-packages\ipykernel\ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


training time this epoch:2.0539748668670654
training loss:1.7366298251199623
training rmse:0.13178125151628975
Training_R_Squared:0.5319227355123227
Epoch number: 40/200
LR: 5e-05
Batch: 0
training time this epoch:1.9896676540374756
training loss:1.4798708561152267
training rmse:0.12164994270920257
Training_R_Squared:0.6011274848510166
Epoch number: 41/200
LR: 5e-05
Batch: 0
training time this epoch:1.722649335861206
training loss:1.3940404590621256
training rmse:0.11806949051563345
Training_R_Squared:0.6242615189333358
Epoch number: 42/200
LR: 5e-05
Batch: 0
training time this epoch:1.9715509414672852
training loss:1.6165244993026135
training rmse:0.12714261674602317
Training_R_Squared:0.5642949571925684
Epoch number: 43/200
LR: 5e-05
Batch: 0
training time this epoch:1.8436689376831055
training loss:1.5072858097902895
training rmse:0.12277156876859925
Training_R_Squared:0.5937382810897722
Epoch number: 44/200
LR: 5e-05
Batch: 0
training time this epoch:1.9905986785888672
training los